# Implementing Various Operations and Algorithms using Tensorflow and Pytorch

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import torch

## Broadcasting 

In [ ]:
x = tf.constant([1,2,3,4])
y = tf.broadcast_to(x, [4,4])
print(K.eval(x))
print('Brodcasted to 4 * 4')
print(K.eval(y))

[1 2 3 4]
Brodcasted to 4 * 4
[[1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]]


In [ ]:
print(K.eval(tf.broadcast_to(tf.constant([1, 2, 3]), [3, 3])))

[[1 2 3]
 [1 2 3]
 [1 2 3]]


##String Tensors

In [ ]:
string_tensor = tf.constant(['This is', 'a', 'String Tensor'])
print(K.eval(string_tensor))

[b'This is' b'a' b'String Tensor']


In [ ]:
#String to Number Conversion
string = tf.constant(['1' '2' '3' '4'])
numbers = tf.strings.to_number(tf.strings.split(string, " "))
print(K.eval(string))
print(numbers)

[b'1234']
tf.RaggedTensor(values=Tensor("StringToNumber_9:0", shape=(?,), dtype=float32), row_splits=Tensor("StringSplit_9/RaggedFromValueRowIds/RowPartitionFromValueRowIds/concat:0", shape=(2,), dtype=int64))


## Sparse Tensors

In [ ]:
sparse_tensor = tf.sparse.SparseTensor(indices=[[0, 0], [1, 1], [2, 2], [3, 3]], values=[1, 2, 3, 4], dense_shape=[4, 4])
dense_tensor = tf.sparse.to_dense(sparse_tensor)

print('Sparse Tensor -->', sparse_tensor)
print('Dense Tensor -->', K.eval(dense_tensor))

Sparse Tensor --> SparseTensor(indices=Tensor("SparseTensor_11/indices:0", shape=(4, 2), dtype=int64), values=Tensor("SparseTensor_11/values:0", shape=(4,), dtype=int32), dense_shape=Tensor("SparseTensor_11/dense_shape:0", shape=(2,), dtype=int64))
Dense Tensor --> [[1 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 4]]


## Ragged Tensors

In [ ]:
list1 = [
    [0, 1, 2, 3],
    [4, 5, 6],
    [7, 8],
    [9]]
ragged_tensor = tf.ragged.constant(list1)
print(ragged_tensor)

tf.RaggedTensor(values=Tensor("RaggedConstant_20/values:0", shape=(10,), dtype=int32), row_splits=Tensor("RaggedConstant_20/Const:0", shape=(5,), dtype=int64))


In [ ]:
print(ragged_tensor.shape)

(4, ?)


## Named Tensors

In [ ]:
torch.zeros(2, 3, names=('N', 'C'))

tensor([[0., 0., 0.],
        [0., 0., 0.]], names=('N', 'C'))

In [ ]:
imgs = torch.randn(1, 2, 2, 3 , names=(None, 'C', 'H', 'W'))
imgs.names

(None, 'C', 'H', 'W')

## Linear Search using Tensorflow

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
def linearSearch(x, array):
  val = tf.constant(x)
  arr = tf.constant(array)
  length = len(array)

  def condition(i, _, is_found):
    return tf.logical_and(tf.less(i, length), tf.logical_not(is_found))

  def body(i, _, is_found):
    return tf.cond(tf.equal(arr[i], val), lambda: (i, arr[i], True), lambda: (tf.add(i, 1), -1, False))

  graph = tf.while_loop(condition, body, [0, val, False],back_prop=False)
  with tf.Session() as sess:
    tf.global_variables_initializer().run()
    return sess.run(graph)


if __name__ == '__main__':
    array, x = [2,3,4,5], 3
    ix, xx, is_found = linearSearch(x, array)
    print('Array :', array)
    print('Number to search :', x)
    if is_found:
        print('{} is at index {}.'.format(xx, ix))
    else:
        print('Not found.')

Array : [2, 3, 4, 5]
Number to search : 3
3 is at index 1.


## Insertion Sort using Tensorflow

In [ ]:
def insertionSort(arr):
  def outer_loop(i, j, _):
    j = i
    cond = lambda i, j, array: tf.logical_and(tf.greater(j,0), tf.greater(array[j-1], array[j]))

    loop = tf.while_loop(cond, inner_loop, loop_vars=[i, j, array],
                shape_invariants=[i.get_shape(), j.get_shape(), tf.TensorShape(length)],
                parallel_iterations=1,
                back_prop=False)
    return tf.add(i, 1), loop[1], loop[2]

  def inner_loop(i, j, _):
    return i, tf.subtract(j, 1), tf.scatter_nd_update(array, [[j-1],[j]], [array[j],array[j-1]])

  length = len(arr)
  start = tf.constant(1)
  end = tf.constant(length - 1)
  array = tf.Variable(arr, trainable=False)


  cond = lambda i, j, _: tf.less(i-1, length-1)
  graph = tf.while_loop(cond, outer_loop, loop_vars=[start, end, array],
          shape_invariants=[start.get_shape(), end.get_shape(), tf.TensorShape(length)],
          parallel_iterations=1,
          back_prop=False)
  with tf.Session() as sess:
    tf.global_variables_initializer().run()
    return sess.run(graph)[2]

with tf.Session() as sess:
    x = np.array([3,2,4,1,5])
    print(x)
    print(insertionSort(x))
    y = np.random.rand(10)
    print(y)
    print(insertionSort(y))


[3 2 4 1 5]
[1 2 3 4 5]
[0.01028168 0.56530693 0.24985666 0.68379803 0.76608574 0.24603765
 0.83205618 0.43248453 0.7621806  0.6460065 ]
[0.01028168 0.24603765 0.24985666 0.43248453 0.56530693 0.6460065
 0.68379803 0.7621806  0.76608574 0.83205618]


## Binary Search using Tensorflow

In [ ]:
def binarySearch(arr, x):
  array = tf.constant(arr)
  val = tf.constant(x)
  length = len(arr)
  def condition(x, is_found, left, right, mid):
    return tf.logical_and(tf.less_equal(left, right), tf.logical_not(is_found))
  def body(x, is_found, left, right, mid):
      mid = tf.to_int32(tf.divide(tf.add(left, right), 2))
      return tf.cond(tf.equal(array[mid],val),
                  lambda: (array[mid], True, left, right, mid),
                  lambda: tf.cond(tf.less(array[mid],val),
                              lambda: (-1, False, tf.add(mid, 1), right, mid),
                              lambda: (-1, False, left, tf.subtract(mid, 1), mid)))

  loop = tf.while_loop(condition, body, [-1,False,0,length,-1],back_prop=False)
  with tf.Session() as sess:
    tf.global_variables_initializer().run()
    return sess.run(loop)


array = sorted([1, 7, 3, 8, 5])
x = 8
xx, is_found, l, r, m = binarySearch(array, x)

print('Array :', array)
print('Number to search :', x)
if is_found:
    print('{} is at index {}.'.format(xx, m))
else:
    print('Not found.')

Array : [1, 3, 5, 7, 8]
Number to search : 8
8 is at index 4.
